# REACT SNIPPETS ⚛️
---

### Use `assert` to fail early

The assert function acts as a safety net. It prevents the app from being built if the NEXT_PUBLIC_API_BASE_URL env variable is missing. So we immediately know that something is wrong before the code is deployed. Note that Next.js ships `dotenv` out of the box

read more: https://dev.to/jkettmann/separate-api-layers-in-react-apps-6-steps-towards-maintainable-code-4n2

In [ ]:
// api/axios.ts

import assert from "assert";
import Axios from "axios";

assert(
  process.env.NEXT_PUBLIC_API_BASE_URL,
  "env variable not set: NEXT_PUBLIC_API_BASE_URL"
);

export const axios = Axios.create({
  baseURL: process.env.NEXT_PUBLIC_API_BASE_URL,
});


// .env

NEXT_PUBLIC_API_BASE_URL=https://prolog-api.profy.dev/v2

---
### decouple auth logic from fetch functions

we can use axios request interceptors to remove headers from fetch request

```
axios.get("/endpoint", {
    params: { page, status: 'open'},
    signal: options.signla,
    headers: {Authoriztion: "my-access-token"} ❗️
})
```

In [ ]:
// api/axios.ts

import assert from "assert";
import Axios, { AxiosRequestConfig } from "axios";

assert(
  process.env.NEXT_PUBLIC_API_BASE_URL,
  "env variable not set: NEXT_PUBLIC_API_BASE_URL"
);

assert(
  process.env.NEXT_PUBLIC_API_TOKEN,
  "env variable not set: NEXT_PUBLIC_API_TOKEN"
);

function authRequestInterceptor(config: AxiosRequestConfig) {
  config.headers.authorization = process.env.NEXT_PUBLIC_API_TOKEN;
  return config;
}

export const axios = Axios.create({
  baseURL: process.env.NEXT_PUBLIC_API_BASE_URL,
});

axios.interceptors.request.use(authRequestInterceptor);

We already achieved a good separation between the fetch functions and the query hooks. As mentioned, the query hooks don’t have any knowledge about Axios being used or about request details like the endpoints.

But the fetch functions themselves aren’t decoupled from Axios yet. We directly export the Axios client and use it in the fetch functions.

In [ ]:
// api/axios.ts

export const axios = Axios.create({
    baseURL: process.env.NEXT_PUBLIC_API_BASE_URL,
  });
  

// api/issues.ts

import { axios } from "./axios";

export async function getIssues(...) {
  const { data } = await axios.get(ENDPOINT, {
    params: { page, ...filters },
    signal: options?.signal,
  });
  return data;
}

If we wanted to replace Axios with something else we’d have to adjust all the fetch functions as well.

To create a further isolation layer we can simply wrap the axios client and only expose its methods indirectly.

In [ ]:
// api/api-client.ts

const axios = Axios.create({
    baseURL: process.env.NEXT_PUBLIC_API_BASE_URL,
  });
  
  export const apiClient = {
    get: (route, config) =>
      axios.get(route, { signal: config?.signal, params: config?.params }),
    post: (route, data, config) =>
      axios.post(route, data, { signal: config?.signal }),
    put: (route, data, config) =>
      axios.put(route, data, { signal: config?.signal }),
    patch: (route, data, config) =>
      axios.patch(route, data, { signal: config?.signal }),
  };

Note that we don’t pass the config object directly to axios. Otherwise, the fetch functions could use any config option that Axios supports. And that again would couple them to Axios.

The fetch function basically would stay the same.

In [ ]:
// api/issues.ts

import { apiClient } from "./api-client";

export async function getIssues(...) {
  const { data } = await apiClient.get(ENDPOINT, {
    params: { page, ...filters },
    signal: options?.signal,
  });
  return data;
}

React tip: If state values are related, and must be kept in sync, create the "derived" value on render.

Example: If you have data and filteredData, create filteredData on render.

Doing so eliminates a useEffect, and needless syncing code. This reduces the chance of bugs.

In [ ]:
//! AVOID: routes and filteredRoutes need to be kept in sync 👎
const [routes, setRoutes] = useState([])
const [filteredRoutes, setFilteredRoutes] = useState([])
const [search, setSearch] = useState("")

// Using useEffect to sync related stated
useEffect(() => {
    setFilteredRoutes(routes.filter((route) => {
        // filtering logic here
    }))
}, [search])

//* PREFERED: Derive related state on render. This eliminates the need for useEffect 👍
const [routes2, setRoutes2] = useState([])
const [search2, setSearch2] = useState("")

// Derive filtered routes on each render. 
// If this is expensive, memoize it.
const filteredRoutes2 = routes.filter((route) => {
            // filtering logic here
})

![](derived_state_tip.png)

---